In [1]:
import pickle
from collections import Counter
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from multiprocessing import Pool
import tqdm
import time

In [3]:
def process(th_num):
    cnt = 0
    n_threads = 1
    lemmatizer = Mystem()
    stemmer = SnowballStemmer('russian', ignore_stopwords=True)
    stop = stopwords.words('russian')
    
    start_time = time.time ()
    
    with open ("../data/docs.tsv") as f:
        for i, line in tqdm.tqdm (enumerate (f), total=582167):
            if i % n_threads != th_num:
                continue
            res = line.decode('utf-8').strip ().split('\t')
            idx = res[0]
            
            title = ""
            text = ""
            
            if len (res) > 1:
                title = res[1]
            if len (res) > 2:
                text = res[2]
            
            with open ("../temp/title_"+idx, "wb") as fout:
                counts = Counter(lemmatizer.lemmatize(title.lower()))
                ndc = filter(lambda x: not (x[0] in stop), counts.items())
                title = dict(map(lambda x: (stemmer.stem(x[0]), x[1]), ndc))
                pickle.dump (title, fout)


            with open ("../temp/body_"+idx, "wb") as fout:
                counts = Counter(lemmatizer.lemmatize(text.lower()))
                ndc = filter(lambda x: not (x[0] in stop), counts.items())
                body = dict(map(lambda x: (stemmer.stem(x[0]), x[1]), ndc))
                pickle.dump (body, fout)
                
            if i/n_threads % 1000 == 0:
                print "thread ",th_num, (i/n_threads) / 582167. * 100,  "%", "elapsed: ", time.time () - start_time 

In [4]:
process (0)

  0%|          | 1/582167 [00:00<114:31:21,  1.41it/s]

thread  0 0.0 % elapsed:  0.709791898727


  0%|          | 209/582167 [01:50<85:37:36,  1.89it/s]

KeyboardInterrupt: 

In [ ]:
pool = Pool (12)

In [ ]:
pool.map (process, range(12))

In [ ]:
with open ("../data/docs.tsv") as f:
    params = [('', '', '') for i in range(n_threads)]
    for line in f:
        if cnt % n_threads == 0:
            Parallel(n_jobs=n_threads)(delayed(process)(*params[j]) for j in range(n_threads))

        idx, title, text = line.decode('utf-8').split('\t')
        params[cnt % n_threads] = (idx, title, text)
        cnt += 1

In [ ]:
def process2(idx, fnames):
    for fname, cnt in zip(fnames, range(len(fnames))):
        if (cnt + 1) % 15000 == 0:
            print(idx, 'finished', cnt)
        dc = {}
        with open('./data/' + fname, 'rb') as fin:
            dc = pickle.load(fin)


        with open('./data/' + fname, 'wb') as fout:
            pickle.dump(ndc, fout)